In [1]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import pyepo
from pyepo.model.grb import optGrbModel
import torch
from torch import nn
from torch.utils.data import DataLoader
import time as time
import torch.optim as optim
import torch.nn as nn
from torch.optim.lr_scheduler import ReduceLROnPlateau, StepLR
from ModelsKnapSack import KnapSackNet, ValPredictNet
from generate_knapsack_data import Gen_Knapsack_data
from knapsack_utils import RegretLoss
from Trainer import trainer
import matplotlib.pyplot as plt

# Set the random seed
seed = 42
torch.manual_seed(seed)

Auto-Sklearn cannot be imported.


In [2]:
# Data parameters
num_knapsack = 2
num_item = 10
num_feat = 10
num_data = 1000
# Load Data
state = Gen_Knapsack_data(num_data, num_feat, num_item, num_knapsack)

Generating training data for knapsack problem with 2 knapsacks and 10 items
Restricted license - for non-production use only - expires 2024-10-28
Optimizing for optDataset...


100%|███████████████████████████████████████| 800/800 [00:00<00:00, 1511.76it/s]


Optimizing for optDataset...


100%|███████████████████████████████████████| 100/100 [00:00<00:00, 1011.77it/s]


Optimizing for optDataset...


100%|████████████████████████████████████████| 100/100 [00:00<00:00, 972.06it/s]

Finished building dataset


In [3]:
weights_numpy = state["weights_numpy"]
capacities = state["capacities"]
dataset_train = state["dataset_train"]
dataset_test = state["dataset_test"]
contexts_numpy = state["contexts_numpy"]

In [4]:
# Initialize DYS_net
device = 'mps'
weights = torch.Tensor(weights_numpy).to(device)
capacities_DYS = capacities.to(device)
DYS_net = KnapSackNet(weights, capacities_DYS, num_knapsack, num_item, num_feat, device=device)
DYS_net.to(device)

/Users/danielmckenzie/My-Drive/Research/Fixed_Point_Networks/Diff-Opt-Over-Polytopes-Project/SPO_with_DYS/source/Knapsack/dYS_opt_net.py:29: UserWarning: The operator 'aten::linalg_svd' is not currently supported on the MPS backend and will fall back to run on the CPU. This may have performance implications. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/mps/MPSFallback.mm:11.)
  U, s, VT = torch.linalg.svd(self.A, full_matrices=False)


KnapSackNet(
  (fc_1): Linear(in_features=10, out_features=1000, bias=True)
  (fc_2): Linear(in_features=1000, out_features=1000, bias=True)
  (fc_3): Linear(in_features=1000, out_features=10, bias=True)
  (leaky_relu): LeakyReLU(negative_slope=0.1)
  (dropout): Dropout(p=0.3, inplace=False)
)

In [5]:
# Train
max_epochs = 20
learning_rate = 1e-3
val_loss_hist, epoch_time_hist, best_test_loss, time_till_best_val_loss = trainer(DYS_net, dataset_train, dataset_test, dataset_test, num_item,
                         num_knapsack, max_epochs, learning_rate, model_type = "DYS",
                         device=device)

Initial validation loss is  0.27018412947654724
epoch:  1 validation loss is  0.1253541111946106 epoch time:  1.4141261577606201
epoch:  2 validation loss is  0.1253541111946106 epoch time:  1.9979557991027832
epoch:  3 validation loss is  0.1253541111946106 epoch time:  2.3797860145568848
epoch:  4 validation loss is  0.1253541111946106 epoch time:  2.3984780311584473
epoch:  5 validation loss is  0.1253541111946106 epoch time:  2.3953583240509033
epoch:  6 validation loss is  0.1253541111946106 epoch time:  2.404285192489624
epoch:  7 validation loss is  0.1253541111946106 epoch time:  2.3834598064422607
epoch:  8 validation loss is  0.1253541111946106 epoch time:  2.322476863861084
epoch:  9 validation loss is  0.1253541111946106 epoch time:  2.374821901321411
epoch:  10 validation loss is  0.1253541111946106 epoch time:  2.3892669677734375


KeyboardInterrupt: 

# Preparing the benchmark approaches

In [ ]:
torch.manual_seed(seed)
val_net = ValPredictNet(num_knapsack, num_item, num_feat, weights,
                        capacities, device=device)
val_net.to(device)

In [ ]:
test_loss_hist_2stage, epoch_time_hist_2stage, best_test_loss_2stage = trainer(val_net, dataset_train, dataset_test, num_item,
                         num_knapsack, max_epochs, learning_rate, 
                         model_type = "Two-stage", device=device)

## Implementing the SPO+ Approach

In [ ]:
torch.manual_seed(seed)
val_net2 = ValPredictNet(num_knapsack, num_item, num_feat, weights,
                        capacities, device=device)
val_net2.to(device)

In [ ]:
torch.manual_seed(seed)
test_loss_hist_SPO, epoch_time_hist_SPO, best_test_loss_SPO = trainer(val_net2, dataset_train, dataset_test, num_item,
                         num_knapsack, max_epochs, learning_rate, 
                         model_type = "SPO+", device=device)

## Differentiable Blackbox Optimization Approach


In [ ]:
torch.manual_seed(seed)
val_net3 = ValPredictNet(num_knapsack, num_item, num_feat, weights,
                        capacities, device=device)
val_net3.to(device)

In [ ]:
test_loss_hist_BBOpt, epoch_time_hist_BBOpt, best_test_loss_SPO = trainer(val_net3, dataset_train, dataset_test, num_item,
                         num_knapsack, max_epochs, learning_rate, 
                         model_type = "BBOpt", device=device)

## Perterbed Optimizer Approach

In [ ]:
torch.manual_seed(seed)
val_net4 = ValPredictNet(num_knapsack, num_item, num_feat, weights,
                        capacities, device=device)
val_net4.to(device)

In [ ]:
test_loss_hist_PertOpt, epoch_time_hist_PertOpt, best_test_loss_PertOpt = trainer(val_net4, dataset_train, dataset_test, num_item,
                         num_knapsack, max_epochs, learning_rate, 
                         model_type = "PertOpt", device=device)

## Fenchel-Young Approach

In [ ]:
torch.manual_seed(seed)
val_net5 = ValPredictNet(num_knapsack, num_item, num_feat, weights,
                        capacities, device=device)
val_net5.to(device)

In [ ]:
test_loss_hist_PertOpt_FY, epoch_time_hist_PertOpt_FY, best_test_loss_FY = trainer(val_net5, dataset_train, dataset_test, num_item,
                         num_knapsack, max_epochs, learning_rate, 
                         model_type = "PertOpt-FY", device=device)

In [ ]:
plt.plot(test_loss_hist_DYS, label='DYS')
plt.plot(test_loss_hist_2stage, label='Two stage')
plt.plot(test_loss_hist_SPO, label='SPO+')
plt.plot(test_loss_hist_BBOpt, label="BBOpt")
plt.plot(test_loss_hist_PertOpt, label="PertOpt")
plt.plot(test_loss_hist_PertOpt_FY, label="PertOpt-FY")
plt.legend()
plt.title("Regret vs epochs")
plt.savefig('Knapsack_regret.png')
plt.show()

In [ ]:
print(learning_rate)

In [ ]:
plt.plot(np.cumsum(epoch_time_hist_DYS), label='DYS')
plt.plot(np.cumsum(epoch_time_hist_2stage), label='Two stage')
plt.plot(np.cumsum(epoch_time_hist_SPO), label='SPO+')
plt.plot(np.cumsum(epoch_time_hist_BBOpt), label='BBOpt')
plt.plot(np.cumsum(epoch_time_hist_PertOpt), label='PertOpt')
plt.plot(np.cumsum(epoch_time_hist_PertOpt_FY), label='PertOpt-FY')
plt.legend()
plt.title("Cumulative Training Time vs Epochs")
plt.savefig('Knapsack_train_time.png')
plt.show()